## Import Modules

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv 
import time
import urllib.request # The urllib.request module defines functions and classes which help in opening URLs (mostly HTTP)
import re
from urllib.request import urlopen
import numpy as np

## Part 1: Obtain a list of dataset pages and query the discovery metadata

We use PASDA's search feature to return a page (https://www.pasda.psu.edu/uci/SearchResults.aspx?Keyword=+) that lists all of the activate dataset landing pages with some metadata. Then, we use the Beautiful Soup module to query this page and harvest the following values:

- Title
- Date Issued
- Publisher
- Description
- Metadata file link
- Download link

### Obtain the file using ONE of the following options

In [89]:
# Option 1: Open "https://www.pasda.psu.edu/uci/SearchResults.aspx?Keyword=+" in a browser and download the page. 
# Save the file as "pasda-search.html" in the same directory as this notebook
# This is recommended if you are testing or running more than once
        
file_path = 'pasda-search.html'  # Modify this to the correct path to your downloaded HTML file
with open(file_path, 'r', encoding='utf-8') as file:
    html_content = file.read()
    
# Option 2: Just read the file directly online. This option is fine if the script is working.
    
# url = 'https://www.pasda.psu.edu/uci/SearchResults.aspx?Keyword=+'
# response = requests.get(url)
# html_content = response.content

In [90]:
soup = BeautifulSoup(html_content, 'html.parser')

# Assuming every dataset is contained in its own <tr> tag
datasets = soup.select('tr[align="left"]')

data = []

dataset_entries = soup.select('td > h3 > a[href^="DataSummary.aspx?dataset="]')

for entry in dataset_entries:
    publisher = entry.find_previous("td").text.strip()
    date = entry.find_previous("td").find_previous("td").text.strip()
    title = entry.text.strip()
    description = entry.find_next("span", id=lambda x: x and x.startswith('DataGrid1_Label3_')).text.strip()
#     metadataLink = entry.find_next_sibling("span").find('a', href=True, string='Metadata')['href']
    metadataFile = entry.parent.parent.find('a', href=True, string='Metadata')['href']
    metadataLink = "https://www.pasda.psu.edu/uci/" + metadataFile 
    try:
        download = entry.parent.parent.find('a', href=True, string='Download')['href']
    except:
        download = ''
        
    # obtain full landing page and create ID
    landing_page = "https://www.pasda.psu.edu/uci/" + entry['href']  # Landing page URL
    iden = 'pasda-' + landing_page.rsplit("=",1)[1]

    data.append([publisher, date, title, description, metadataFile, metadataLink, download, landing_page, iden])
    

# Convert to pandas dataframe
import pandas as pd
df = pd.DataFrame(data, columns=['Publisher', 'Date', 'Title', 'Description', 'Metadata File', 'HTML', 'Download', 'Information', 'ID'])
    

In [91]:
# optional: check to see that the metadata is parsing into the dataframe correctly
actionDate = time.strftime('%Y%m%d')
df.to_csv(f'output_{actionDate}.csv', index=False)

# Part 2: Query the supplemental metadata

Context: Most of the records have supplemental metadata in ISO 19139 or FGDC format. The link to this document is found in the 'HTML" column. Although these files are created as XMLs, the link is a rendered HTML.

There is additional information in these files that we want to scrape, including bounding boxes and geometry type.

We will start by downloading the metadata files - this will save time and reduce the load on PASDA's servers because this part of the recipe may need to be run multiple times after troubleshooting.

### Download the metadata files to a folder called "metadata_files"

In [81]:
import os

# Create a directory named 'metadata_files' to store the downloaded files
download_folder = 'metadata_files'
if not os.path.exists(download_folder):
    os.makedirs(download_folder)

In [83]:
import requests

def download_file(url, folder):
    """
    Download a file given its URL and store it in the specified folder.
    """
    # Get the filename from the URL
    filename = url.split("/")[-1]
    response = requests.get(url, stream=True)
    
    # Handle the response's content in chunks (useful for large files)
    with open(os.path.join(folder, filename), 'wb') as file:
        for chunk in response.iter_content(chunk_size=8192):
            file.write(chunk)

In [84]:
for url in df['HTML']:
    try:
        download_file(url, download_folder)
        print(f"Downloaded {url}")
    except Exception as e:
        print(f"Error downloading {url}. Reason: {e}")

Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=AdamsCounty_AgSecurityAreas202306.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=AdamsCounty_LandConservancyEasements202306.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=AdamsCounty_MtJoyTwpPreservedFarms202306.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=AdamsCounty_MunicipalBoundary202306.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=AdamsCounty_Parcels202306.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=AdamsCounty_PreservedFarms202306.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=AdamsCounty_Roads202306.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=NWPAGlacialDeposits2018.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=AlleghenyCollege_GlacialDeposits1979.xml
Downloaded https://www.pasd

Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=batBigBrown.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=batEasternPipistrelle.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=batEasternRed.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=batEasternSmallfootedMyotis.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=batEveningBat.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=batHoary.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=batIndianaMyotis.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=batLittleBrownMyotis.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=batNorthernMyotis.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=batRafinesquBigeared.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=bat

Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=ChesterCounty_Bridgepoints201306.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=ChesterCounty_BuildingFootprints2015.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=ChesterCounty_Cemeteries201306.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=ChesterCounty_CountyBoundary201107.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=ChesterCounty_ElectionPollingPlaces202006.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=ChesterCounty_Highereducationsfacilities201306.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=ChesterCounty_MuniBoundaries202006.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=ChesterCountyParcels202006.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=ChesterCounty_Railroads201306.xml
Download

Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=PhillyAirport_Amenities.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=PhillyAirport_Art.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=PhillyAirport_Parking_Entry_Exit.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=PhillyAirport_Parking_Lots.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=PhillyAirport_Rental_Car_Locations.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=PhillyAirport_Rental_Car_Lots.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=PhillyAirport_Roadway_Zones.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=PhillyAirport_Runways.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=PhillyAirport_Terminal_Buildings.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=Phil

Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=PhillyPlanning_Political_Wards.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=PhillyPlanning_Schools.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=PhillyPlanning_Zoning_BaseDistricts.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=PhillyPlanning_Zoning_RCO.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=philadelphiaplanninganalysissections2005.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=PhillyPolice_Boundaries_District.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=PhillyPolice_Boundaries_Division.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=PhillyPolice_Boundaries_PSA.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=PhillyPolice_CCD_BOUNDARY.xml
Downloaded https://www.pasda.psu.edu/uci/FullMet

...

Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=NLCD_2017_Tree_Canopy_Cover_PA_v2021-4.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=NLCD_2018_Tree_Canopy_Cover_PA_v2021-4.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=NLCD_2019_Tree_Canopy_Cover_PA_v2021-4.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=NLCD_2020_Tree_Canopy_Cover_PA_v2021-4.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=NLCD_2021_Tree_Canopy_Cover_PA_v2021-4.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=impervious14_southeast3k_091406.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=canopy13_pa_northeast_3k_022007.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=canopy14_southeast_3k_022007.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=impervious13_pa_northeast_010407.xml
Down

Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=Western_PA_QL1North_ClassifiedPointCloud2020.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=Western_PA_QL1North_Contour2020.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=Western_PA_QL1North_DEM2020.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=Western_PA_QL1North_Intensity2020.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=Western_PA_QL2North_Breakline2020.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=Western_PA_QL2South_Breakline2020.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=Western_PA_QL2North_ClassifiedPointCloud2020.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=Western_PA_QL2South_ClassifiedPointCloud2020.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=Western_PA_QL2North_Contour2020.

Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=AllentownPA_Orthos2016.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=AllentownPA_Orthos2016.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=USGS_LiDAR2017.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=USGS_LiDAR2017.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=USGS_LiDAR2017.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=USGS_LiDAR2017.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=USGSLiDAR_DauphinCoPA_2016.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=USGSLiDAR_DauphinCoPA_2016.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=USGSLiDAR_DauphinCoPA_2016.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=USGSLiDAR_DauphinCoPA_2016.xml
Downloaded https://www.pasda.psu.edu

Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=DRB_ProtectedLands202307.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=DRB_ProtectedLands202307.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=DRB_ProtectedLands202307.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=DRB_ProtectedLands202307.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=NHD_HR_Stream_Order_PA.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=AgriculturalSecurityAreas202011.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=ConservationEasements202307.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=FarmEasements202306.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=Federal202306.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=Local202306.xml
Downloaded https://ww

Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=YorkCounty_PUB_FACIL_Schools202209.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=YorkCounty_DIST_Senate202209.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=Pennsylvania_soil_metadata_201409.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=YorkCounty_HYDRO_Streams202209.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=YorkCounty_Transit_Routes202209.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=YorkCounty_TRANSP_Transit_Stops202209.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=YorkCounty_ENVIR_Unique_Features202209.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=YorkCounty_DIST_Voting202209.xml
Downloaded https://www.pasda.psu.edu/uci/FullMetadataDisplay.aspx?file=YorkCounty_Zipcodes202209.xml
Downloaded https://www.pasda.psu

### Query the downloaded files

In [103]:
bounding_boxes = {}
spatial_reference_methods = {}
theme_keywords_list = []
place_keywords_list = []

metadata_folder = "metadata_files"

for metadata_file in df['Metadata File']:
    file_path = os.path.join(metadata_folder, metadata_file)
    
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            file_content = file.read()
            soup = BeautifulSoup(file_content, "html.parser")

    # for metadata_url in df['HTML']:
    #     response = requests.get(metadata_url)
    #     soup = BeautifulSoup(response.content, "html.parser")
            try: 
                try:
                    west = soup.find('i', string='West_Bounding_Coordinate:').next_sibling.strip()   
                except AttributeError:
#                     print(f"'West_Bounding_Coordinate:' not found in {metadata_file}")
                    west = '-80.52'

                try:
                    south = soup.find('i', string='South_Bounding_Coordinate:').next_sibling.strip()   
                except AttributeError:
#                     print(f"'South_Bounding_Coordinate:' not found in {metadata_file}")
                    south = '39.72'

                try:
                    east = soup.find('i', string='East_Bounding_Coordinate:').next_sibling.strip()   
                except AttributeError:
#                     print(f"'East_Bounding_Coordinate:' not found in {metadata_file}")
                    east = '-74.69'

                try:
                    north = soup.find('i', string='North_Bounding_Coordinate:').next_sibling.strip()   
                except AttributeError:
#                     print(f"'North_Bounding_Coordinate:' not found in {metadata_file}")
                    north = '42.51'

                bbox = west + ',' + south + ',' +east + ',' + north
        
#                 try:
#                     direct_spatial_ref = soup.find('i', string='Direct_Spatial_Reference_Method:').next_sibling.strip()
#                 except AttributeError:
#                     print(f"'Direct_Spatial_Reference_Method:' not found in {metadata_file}")
#                     direct_spatial_ref = 'unspecified'
            except:
                bbox = "-80.52,39.72,-74.69,42.51"

            
#             keyword_values = ""
    
#             try:
#             # Extract all instances of Theme_Keyword
#                 theme_keywords = soup.findAll('i', string='Theme_Keyword:')
        
#             # Get the values following each Theme_Keyword and concatenate them with "|"
#                 keyword_values = "|".join([kw.next_sibling.strip() for kw in theme_keywords])
#             except Exception as e:
#                 print(f"No theme keywords for {metadata_file}: {e}")
    
#                 theme_keywords_list.append(keyword_values if keyword_values else " ")


#             place_values = ""
    
#             try:
#             # Extract all instances of Place_Keyword
#                 place_keywords = soup.findAll('i', string='Plcae_Keyword:')
        
#             # Get the values following each Theme_Keyword and concatenate them with "|"
#                 place_values = "|".join([kw.next_sibling.strip() for kw in place_keywords])
#             except Exception as e:
#                 print(f"No place keywords for {metadata_file}: {e}")
    
#                 place_keywords_list.append(place_values if place_values else " ")
                
                
                

            bounding_boxes[metadata_file] = bbox
            spatial_reference_methods[metadata_file] = direct_spatial_ref

    except:
        pass

df['Bounding Box'] = df['Metadata File'].map(bounding_boxes)
# df['Resource Type'] = direct_spatial_ref + ' data'
# df['Resource Class'] = np.where(df['Resource Type'] == 'Raster data', 'Imagery', 'Datasets')
# df['Keyword'] = theme_keywords_list
# df['Spatial Coverage'] = place_keywords_list


actionDate = time.strftime('%Y%m%d')
df.to_csv(f'output_{actionDate}.csv', index=False)
print('#### Job done ####')

'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=AdamsCounty_AgSecurityAreas202306.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=AdamsCounty_LandConservancyEasements202306.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=AdamsCounty_MtJoyTwpPreservedFarms202306.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=AdamsCounty_MunicipalBoundary202306.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=AdamsCounty_Parcels202306.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=AdamsCounty_PreservedFarms202306.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=AdamsCounty_Roads202306.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=NWPAGlacialDeposits2018.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=Alleg

'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=batRafinesquBigeared.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=batSeminole.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=batSilverhaired.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=batSoutheasternMyotis.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=batTownsendBigeared.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=BedfordCountyParcels202307.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=BlairCounty_AddressPoints202210.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=BlairCounty_StreetCenterlines202210.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=BlairCounty_Parcels202210.xml
'Direct_Spatial_Reference_Method:' not found in Fu

'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=ChesterCounty_Railroads201306.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=ChesterCounty_RoadCenterlines202006.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=ChesterCounty_Schooldistrictboundaries201306.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=ChesterCounty_Schools201306.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=ChesterCounty_ZipCodes2017.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=LivableLandscapesBasemapFeatures201306.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=LivableLandscapesNaturalResourcesOverlay201306.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=ChoptankRiverHeritage_Access201101.xml
'Direct_Spatial_Reference_Method:' not found in FullMetad

'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=Philadelphia_Building_3DModels.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=Philadelphia_Building_3DModels.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=Philadelphia_Building_3DModels.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=PhiladelphiaBuildings2017.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=PhiladelphiaCensusBlockGroups201201.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=philadelphiacensustracts2000.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=Philadelphia_Census_Tracts_2010_201302.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=PhillyCommerce_Business_Services_Market_Areas.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?fi

'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=PhillyPolice_INCIDENTS_2006.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=PhillyPolice_INCIDENTS_2007.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=PhillyPolice_INCIDENTS_2008.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=PhillyPolice_INCIDENTS_2009.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=PhillyPolice_INCIDENTS_2010.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=PhillyPolice_INCIDENTS_2011.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=PhillyPolice_INCIDENTS_2012.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=PhillyPolice_INCIDENTS_2013.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=PhillyPolice_INCIDENTS_2014.xml
'Direct_Sp

...

'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=NAOGP_GreenbrierLimestoneAssessmentUnitQuarterMileCells.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=maptechhistoric2005.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=maptech_v2.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=NAOGP_MarcellusShaleAssessmentUnit.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=NAOGP_MarcellusShaleAssessmentUnitQuarterMileCells.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=MD_PA_SandyLiDAR2015.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=MD_PA_SandyLiDAR2015.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=MD_PA_SandyLiDAR2015.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=NAOGP_MississippianSandstonesA

'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=PA_SandyLiDAR2015.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=nlcd_2011_landcover_2011_edition_2014_10_10.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=nlcd_2011_impervious_2011_edition_2014_10_10.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=NLCD_2016_Land_Cover_L48.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=NLCD_Land_Cover_Change_Index_L48.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=nlcd_2011_USFS_tree_canopy_2011_edition_2014_03_31.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=NALCMS_2020_PA_Land_Cover_30m.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=NAOGP_NorthwesternOhioShaleAssessmentUnit.xml
'Direct_Spatial_Reference_Method:' not found in FullMeta

'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=USGS_GEOpdf.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=USGS_GEOpdf_Historical.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=USGS_GEOpdf.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=USGS_GEOpdf2014.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=USGS_GEOpdf2020.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=USGS_GEOpdf_Historical.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=USGS_GEOpdf30_60min.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=USGS_GEOpdf.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=USGS_GEOpdf_Historical.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=USGS_GEOpdf_Historical.xml


'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=ForestType_2011_DRBC.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=Baywide_LandCover.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=Baywide_LandCover.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=Baywide_LandCover.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=Baywide_LandCover.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=Baywide_LandCover.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=Baywide_LandCover.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=landcover_2013_delawareriverbasin.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=LandCover_MappingArea_2010_DRBC.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay

'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=WPC_huc12calclu.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=WPC_nonprotectedsites.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=WPC_rf3shedlu.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=WyomingCounty_Addresses202109.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=WyomingCounty_Parcels202109.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=WyomingCounty_Streets202109.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=YorkCounty_Address_Pts202209.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=YorkCounty_Ag_Security202209.xml
'Direct_Spatial_Reference_Method:' not found in FullMetadataDisplay.aspx?file=YorkCounty_TRANSP_Airports202209.xml
'Direct_Spatial_Reference_Method:

## Part 3: add default and calculated values

In [ ]:
# Create Date Range field
df['Date Range'] = df.apply(lambda row: f"{row['Date Issued']}-{row['Date Issued']}" if pd.notna(row['Date Issued']) else '', axis=1)

In [ ]:
# Create an identifier
iden = 'pasda-' + landingPage.rsplit("=",1)[1]

In [ ]:
# Append default values

df['Code'] = '08a-01'
df['Access Rights'] = 'Public'
df['Accrual Method'] = 'HTML'
df['Date Accessioned'] = time.strftime('%Y-%m-%d')
df['Language'] = 'eng'
df['Is Part Of'] = '08a-01'
df['Member Of'] = 'ba5cc745-21c5-4ae9-954b-72dd8db6815a'
df['Provider'] = 'Pennsylvania Spatial Data Access (PASDA)'

In [ ]:
df

In [ ]:
# Define the desired order of columns
desired_order = [
'Title',
'Alternative Title',
'Description',
'Language',
'Creator',
'Publisher',
'Provider',
'Resource Class',
'Resource Type',
'Theme',
'Keyword',
'Date Issued',
'Date Range',
'Spatial Coverage',
'Bounding Box',
'Member Of',
'Download',
'HTML',
'Information',
'ID',
'Identifier',
'Access Rights',

# Add more columns as needed in the desired order
]

# Reindex the DataFrame based on the desired order of columns
df = df.reindex(columns=desired_order)